# Lecture 3-4 LAB: Intensity Transformations

## 0.- Initialize filesystem and libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pydicom

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pydicom

## 1.- Mathematical Mapping Functions

### Negative Transformation (Inverse Image)

In [ ]:
# Load the image from a file in Google Drive
image_path = '/content/drive/MyDrive/PIM/Images/X-ray_1.jpeg'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Compute the inverse of the image using bitwise NOT
inverse_image = cv2.bitwise_not(image)

# Create a figure with a single row and two columns
plt.figure(figsize=(10, 5))

# Plot the original image
plt.subplot(1, 2, 1)
plt.imshow(image, cmap='gray', vmin=0, vmax=255)
plt.title("Original image")
plt.axis('off')

# Plot the inverse image
plt.subplot(1, 2, 2)
plt.imshow(inverse_image, cmap='gray', vmin=0, vmax=255)
plt.title("Inverse image")
plt.axis('off')

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

#### Compute the histogram of the original and inverse images (assigned task)

### Power Law Transformation (Gamma Correction)

In [ ]:
# Load the image from a file in Google Drive
image_path = '/content/drive/MyDrive/PIM/Images/X-ray_1.jpeg'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Define the gamma values
gamma_values = [1/5, 1/2, 1, 2, 5]

# Create a figure with 1 row and 5 columns
plt.figure(figsize=(10, 4))

# Plot each gamma corrected image
for i, gamma in enumerate(gamma_values): # enumerate returns the index and the value
  plt.subplot(1, 5, i + 1)
  # normalize ( / 255.0), apply gamma correction (np.power(...)), and scale back ( * 255)
  corrected_image = np.uint8(np.power(image / 255.0, gamma) * 255)
  plt.imshow(corrected_image, cmap='gray', vmin=0, vmax=255)
  plt.title(f'Gamma = {gamma}')
  plt.axis('off')

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

#### Compute the histograms of the power law transformation images (assigned task)

### Logarithmic and exponential transformations

In [ ]:
# Load the image from a file in Google Drive
image_path = '/content/drive/MyDrive/PIM/Images/X-ray_1.jpeg'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Logarithmic and exponential transformations (with normalization and scaling back)

# np.log1p(x) computes log(1 + x) for numerically stability for small x values
log_image = np.uint8((np.log1p(image) / np.log(256)) * 255)
#log_image = np.uint8((np.log(1 + image) / np.log(256)) * 255)

# np.expm1(x) computes exp(x) - 1 for numerically stability for small values of x
exp_image = np.uint8(np.expm1(image / 255.0 * np.log(256)))
#exp_image = np.uint8(np.exp(image / 255.0 * np.log(256)) - 1)

# Plot the results
plt.figure(figsize=(10, 4))

# Original image
plt.subplot(1, 3, 1)
plt.imshow(image, cmap='gray', vmin=0, vmax=255)
plt.title('Original image')
plt.axis('off')

# Normalized image
plt.subplot(1, 3, 2)
plt.imshow(log_image, cmap='gray', vmin=0, vmax=255)
plt.title('Logarithmic transformation')
plt.axis('off')

# Corrected normalized image
plt.subplot(1, 3, 3)
plt.imshow(exp_image, cmap='gray', vmin=0, vmax=255)
plt.title('Exponential transformation')
plt.axis('off')

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

#### Compute the histograms of the logarithmic and exponential transformations (assigned task)

### Sigmoid correction

In [ ]:
# Load the original image in grayscale
image_path = '/content/drive/MyDrive/PIM/Images/X-ray_1.jpeg'
original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Alpha and Beta values to test
alpha_values = [1, 5, 10]
beta_values = [0.1, 0.5, 1]

# Create a figure with 2 rows and 4 columns
plt.figure(figsize=(12, 6))

# Plot the original image
plt.subplot(2, 4, 1)
plt.imshow(original_image, cmap='gray', vmin=0, vmax=255)
plt.title('Original image')
plt.axis('off')

# Apply sigmoid correction with different alpha values
for i, alpha in enumerate(alpha_values):
    normalized_image = original_image / 255.0
    corrected_image = 1 / (1 + np.exp(-alpha * (normalized_image - 0.5)))  # beta=0.5
    corrected_image = (corrected_image * 255).astype(np.uint8)

    plt.subplot(2, 4, i + 2)
    plt.imshow(corrected_image, cmap='gray', vmin=0, vmax=255)
    plt.title(f'Alpha = {alpha}, Beta = 0.5')
    plt.axis('off')

# Apply sigmoid correction with different beta values
for i, beta in enumerate(beta_values):
    normalized_image = original_image / 255.0
    corrected_image = 1 / (1 + np.exp(-5 * (normalized_image - beta)))  # alpha=5
    corrected_image = (corrected_image * 255).astype(np.uint8)

    plt.subplot(2, 4, i + 6)
    plt.imshow(corrected_image, cmap='gray', vmin=0, vmax=255)
    plt.title(f'Alpha = 5, Beta = {beta}')
    plt.axis('off')

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

#### Compute the histograms of the sigmoid correction (assigned task)

### Piecewise Linear Transformation

In [ ]:
# Load the image in grayscale
image_path = '/content/drive/MyDrive/PIM/Images/X-ray_1.jpeg'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Initialize the transformed image
transformed_image = np.zeros_like(image, dtype=np.uint8)

# Apply the piecewise linear transformation
for i in range(image.shape[0]): # rows
    for j in range(image.shape[1]): # columns
        value = image[i, j]
        if value < 100:
            transformed_image[i, j] = np.uint8(0.5 * value) # convert the new value to uint8 type
        elif value < 150:
            transformed_image[i, j] = np.uint8(3 * (value - 100) + 50)
        else:
            transformed_image[i, j] = np.uint8((55 / 105) * (value - 150) + 200)

# Display the original and transformed images using a simpler subplot format
plt.figure(figsize=(10, 5))

# Original image
plt.subplot(1, 2, 1)
plt.imshow(image, cmap='gray', vmin=0, vmax=255)
plt.title('Original image')
plt.axis('off')

# Transformed image
plt.subplot(1, 2, 2)
plt.imshow(transformed_image, cmap='gray', vmin=0, vmax=255)
plt.title('Piecewise linear transformed image')
plt.axis('off')

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

#### Compute the histogram of the piecewise linear transformation (assigned task)

#### Hard level slicing

In [ ]:
# Load the image in grayscale
image_path = '/content/drive/MyDrive/PIM/Images/X-ray_1.jpeg'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Define parameters for the piecewise linear transformation
I_low, I_high = 50, 120
highlight_value, background_value = 255, 0

# Initialize the transformed image
transformed_image = np.zeros_like(image, dtype=np.uint8)

# Apply the piecewise linear transformation
for i in range(image.shape[0]): # rows
    for j in range(image.shape[1]): # columns
        if I_low <= image[i, j] <= I_high:
            transformed_image[i, j] = highlight_value # highlight the range [I_low, I_high]
        else:
            transformed_image[i, j] = background_value # the rest is set to background_value

# Display the original and transformed images using a simpler subplot format
plt.figure(figsize=(10, 5))

# Original image
plt.subplot(1, 2, 1)
plt.imshow(image, cmap='gray')
plt.title('Original image')
plt.axis('off')

# Transformed image (Intensity Level Sliced)
plt.subplot(1, 2, 2)
plt.imshow(transformed_image, cmap='gray')
plt.title('Hard level sliced image')
plt.axis('off')

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

#### Soft level slicing

In [ ]:
# Load the image in grayscale
image_path = '/content/drive/MyDrive/PIM/Images/X-ray_1.jpeg'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Define parameters for the piecewise linear transformation
I_low, I_high = 50, 120
highlight_value = 255

# Initialize the transformed image
transformed_image = np.zeros_like(image, dtype=np.uint8)

# Apply the piecewise linear transformation
for i in range(image.shape[0]): # rows
    for j in range(image.shape[1]): # columns
        if I_low <= image[i, j] <= I_high:
            transformed_image[i, j] = highlight_value # highlight the range [I_low, I_high]
        else:
            transformed_image[i, j] = image[i, j] # the rest is unaltered

# Display the original and transformed images using a simpler subplot format
plt.figure(figsize=(10, 5))

# Original image
plt.subplot(1, 2, 1)
plt.imshow(image, cmap='gray', vmin=0, vmax=255)
plt.title('Original image')
plt.axis('off')

# Transformed image
plt.subplot(1, 2, 2)
plt.imshow(transformed_image, cmap='gray', vmin=0, vmax=255)
plt.title('Soft level sliced image')
plt.axis('off')

plt.tight_layout()
plt.show()

#### Windowing

In [ ]:
# Function to convert pixel values to Hounsfield Units (HU)
def get_hounsfield_units(dicom_data):
    # Get the pixel array from the DICOM file (pixel_array)
    # DICOM files store pixel data as integers
    image = dicom_data.pixel_array.astype(np.float64) # Convert the pixel array to float64

    # Print the original min and max pixel values
    print("Raw min pixel value:", np.min(image))
    print("Raw max pixel value:", np.max(image))

    # Read the values of the DICOM tags 'RescaleSlope' and 'RescaleIntercept'

    # Get 'RescaleSlope', default to 1 if not found
    if 'RescaleSlope' in dicom_data:
      rescale_slope = dicom_data.RescaleSlope
    else:
      rescale_slope = 1

    # Get 'RescaleIntercept', default to 0 if not found
    if 'RescaleIntercept' in dicom_data:
      rescale_intercept = dicom_data.RescaleIntercept
    else:
      rescale_intercept = 0

    # Print the Rescale Slope and Rescale Intercept
    print("Rescale Slope:", rescale_slope)
    print("Rescale Intercept:", rescale_intercept)

    # Rescale the pixel values using the Rescale Slope and Rescale Intercept attributes
    # These attributes are used to convert the raw pixel values into Hounsfield Units (HU)
    # The conversion is done by applying the formula:
    # HU = RawPixelValue * RescaleSlope + RescaleIntercept
    hu_image = image * rescale_slope + rescale_intercept

    # Return the converted image in Hounsfield Units
    return hu_image

# Function to apply windowing
def apply_windowing(hu_image, HUmin, HUmax):
    # Create a copy of the image to apply windowing
    windowed_image = np.copy(hu_image)
    # windowed_image = hu_image will not work (both variables point to the same array in memory)
    # Any modifications made to windowed_image will also be reflected in hu_image (and vice versa)

    # Apply the windowing by setting values outside the range to HUmin
    # Create a boolean mask for pixel values in hu_image that are less (larger) than HUmin (HUmax)
    # Replace those pixel values in windowed_image with HUmin (HUmax)
    windowed_image[hu_image < HUmin] = HUmin
    windowed_image[hu_image > HUmax] = HUmax
    # The rest of values (HUmin <= hu_image <= HUmax) remain unaltered

    return windowed_image

# Read the DICOM file
dicom_file_path = '/content/drive/MyDrive/PIM/Images/ID_0032_CONTRAST_1_CT.dcm'
dicom_data = pydicom.dcmread(dicom_file_path)

# Convert the pixel values to Hounsfield Units
hu_image = get_hounsfield_units(dicom_data)

# Calculate and print the minimum and maximum HU values
min_HU = np.min(hu_image)
max_HU = np.max(hu_image)
print(f"Minimum HU value: {min_HU}")
print(f"Maximum HU value: {max_HU}")

# Define the HU range for windowing
window_HUmin, window_HUmax = 200, 500

# Apply windowing to the HU image
windowed_image = apply_windowing(hu_image, window_HUmin, window_HUmax)

# Create a 1x2 grid of subplots using plt.subplot
plt.figure(figsize=(10, 5))

# Display the original HU image
plt.subplot(1, 2, 1)
plt.imshow(hu_image, cmap='gray', vmin=min_HU, vmax=max_HU)
plt.title('DICOM image in Hounsfield Units')
plt.axis('off')
cbar = plt.colorbar()
cbar.set_label('Hounsfield Units (HU)')

# Display the windowed HU image
plt.subplot(1, 2, 2)
plt.imshow(windowed_image, cmap='gray', vmin=window_HUmin, vmax=window_HUmax)
plt.title(f'Windowed image [{window_HUmin}, {window_HUmax}] HU')
plt.axis('off')
cbar = plt.colorbar()
cbar.set_label('Hounsfield Units (HU)')

# Adjust layout
plt.tight_layout()
plt.show()

## 2.- Histogram-Based Techniques

### Thresholding

In [ ]:
# Load the original image in grayscale
image_path = '/content/drive/MyDrive/PIM/Images/X-ray_1.jpeg'
original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Set a threshold value
threshold_value = 127

# Apply the threshold
# cv2.threshold is used to convert a grayscale image into a binary image
# Arguments:
# - original_image: input grayscale image
# - threshold_value: threshold value
# - 255: the maximum value assigned to pixels exceeding the threshold
# - cv2.THRESH_BINARY: the type of thresholding applied. THRESH_BINARY sets pixels above
#   the threshold to the max value, and pixels below the threshold to 0.
_, binary_image = cv2.threshold(original_image, threshold_value, 255, cv2.THRESH_BINARY)

# Create the figure
plt.figure(figsize=(8, 8))

# Plot the original image
plt.subplot(2, 2, 1)
plt.imshow(original_image, cmap='gray', vmin=0, vmax=255)
plt.title('Original image')
plt.axis('off')

# Plot the histogram of the original image
plt.subplot(2, 2, 2)
plt.hist(original_image.flatten(), bins=256, range=[0, 256], color='black')
plt.title('Histogram of original image')
plt.xlabel('Pixel intensity')
plt.ylabel('Frequency')

# Plot the binarized image
plt.subplot(2, 2, 3)
plt.imshow(binary_image, cmap='gray', vmin=0, vmax=255)
plt.title('Binary image')
plt.axis('off')

# Plot the histogram of the binary image
plt.subplot(2, 2, 4)
plt.hist(binary_image.flatten(), bins=256, range=[0, 256], color='black')
plt.title('Histogram of binary image')
plt.xlabel('Pixel intensity')
plt.ylabel('Frequency')

# Adjust layout
plt.tight_layout()
plt.show()

### Adaptive Thresholding

In [ ]:
# Load the original image in grayscale
image_path = '/content/drive/MyDrive/PIM/Images/CT_slice_0016.png'
original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Apply the threshold
_, binary_image = cv2.threshold(original_image, 100, 255, cv2.THRESH_BINARY)

# Apply Adaptive Mean Thresholding
# Arguments:
# - original_image: input grayscale image
# - cv2.ADAPTIVE_THRESH_MEAN_C: the adaptive method calculates the threshold value as
#   the mean of the pixel values in the neighborhood area (specified by the block size), minus a constant C
# - cv2.THRESH_BINARY: the type of thresholding to apply (binary)
# - 49: block size (an odd number) that defines the size of the neighborhood around each pixel
#   to calculate the threshold
# - 2: the constant C that is subtracted from the mean value calculated for the neighborhood
adaptive_mean_image = cv2.adaptiveThreshold(
    original_image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 49, 2)

# Apply Adaptive Gaussian Thresholding
# - cv2.ADAPTIVE_THRESH_GAUSSIAN_C: calculates the threshold value as a weighted sum of
#   the pixel values in the neighborhood area (with weights being determined by a Gaussian window),
#   minus a constant C.
adaptive_gaussian_image = cv2.adaptiveThreshold(
    original_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 49, 2)

# Create the figure
plt.figure(figsize=(8, 8))

# Plot the original image
plt.subplot(2, 2, 1)
plt.imshow(original_image, cmap='gray', vmin=0, vmax=255)
plt.title('Original image')
plt.axis('off')

# Plot the binarized image
plt.subplot(2, 2, 2)
plt.imshow(binary_image, cmap='gray', vmin=0, vmax=255)
plt.title('Binary thresholding')
plt.axis('off')

# Plot the image with adaptive mean thresholding
plt.subplot(2, 2, 3)
plt.imshow(adaptive_mean_image, cmap='gray', vmin=0, vmax=255)
plt.title('Adaptive mean\n thresholding')
plt.axis('off')

# Plot the image with adaptive gaussian thresholding
plt.subplot(2, 2, 4)
plt.imshow(adaptive_gaussian_image, cmap='gray', vmin=0, vmax=255)
plt.title('Adaptive Gaussian\n thresholding')
plt.axis('off')

# Adjust layout
plt.tight_layout()
plt.show()

### Contrast Stretching

In [ ]:
# Load the original image in grayscale
image_path = '/content/drive/MyDrive/PIM/Images/CT_slice_0016.png'
original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Minimum and maximum values that include most of the pixel intensities
lower_bound, upper_bound = 85, 125

# Clip values outside the range [lower_bound, upper_bound]
clipped_image = np.clip(original_image, lower_bound, upper_bound)

# Apply contrast stretching using OpenCV's normalize function
# 'original_image' is the input grayscale image
# 'None' indicates that the output image will be created automatically and will not return the result
# 'alpha=lower_bound' sets the lower bound of the new intensity range
# 'beta=upper_bound' sets the upper bound of the new intensity range
# 'norm_type=cv2.NORM_MINMAX' specifies that the normalization should map the
#    minimum and maximum intensity values in the original image to the range [alpha, beta]
mapped_image = cv2.normalize(clipped_image, None, 0, 255, norm_type=cv2.NORM_MINMAX)

# Create a 2x2 grid for plots using plt.subplot
plt.figure(figsize=(8, 8))

# Plot the original image
plt.subplot(2, 2, 1)
plt.imshow(original_image, cmap='gray', vmin=0, vmax=255)
plt.title('Original image')
plt.axis('off')

# Plot the histogram of the original image
plt.subplot(2, 2, 2)
plt.hist(original_image.flatten(), bins=256, range=[0, 256], color='black')
plt.title('Original image histogram')
plt.xlim([0, 256])
plt.ylim(0, 15000)

# Plot the mapped image
plt.subplot(2, 2, 3)
plt.imshow(mapped_image, cmap='gray', vmin=0, vmax=255)
plt.title(f'Image mapped to [{lower_bound}, {upper_bound}]')
plt.axis('off')

# Plot the histogram of the mapped image
plt.subplot(2, 2, 4)
plt.hist(mapped_image.flatten(), bins=256, range=[0, 256], color='black')
plt.title(f'Histogram mapped to [{lower_bound}, {upper_bound}]')
plt.xlim([0, 256])
plt.ylim(0, 15000)

plt.tight_layout()
plt.show()

### Histogram Equalization

In [ ]:
# Load the original image in grayscale
image_path = '/content/drive/MyDrive/PIM/Images/CT_slice_0016.png'
original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Apply histogram equalization
equalized_image = cv2.equalizeHist(original_image)

# Create a 1x2 subplot to compare original and equalized images
plt.figure(figsize=(8, 8))

# Display the original image
plt.subplot(2, 2, 1)
plt.imshow(original_image, cmap='gray', vmin=0, vmax=255)
plt.title('Original image')
plt.axis('off')

# Plot the histogram of the original image (it uses plt.hist)
plt.subplot(2, 2, 2)
# Plot the histogram of the original image
# Flatten the 2D image array into a 1D array of pixel intensities using .flatten()
# This allows us to pass the pixel intensities as a 1D array to the histogram function
plt.hist(                      # Plot the histogram
    original_image.flatten(),  # Flatten the 2D image to 1D
    bins=256,                  # Create 256 bins, one for each possible intensity value (0-255)
    range=[0, 256],            # Set the range of the histogram from 0 to 255 (inclusive)
    color='black'              # Set the color of the histogram bars to black
)
plt.title('Original image histogram')
plt.xlim([0, 256])
plt.ylim(0, 15000)

# Display the equalized image
plt.subplot(2, 2, 3)
plt.imshow(equalized_image, cmap='gray', vmin=0, vmax=255)
plt.title('Equalized image')
plt.axis('off')

# Plot the histogram of the equalized image
plt.subplot(2, 2, 4)
plt.hist(equalized_image.flatten(), bins=256, range=[0, 256], color='black')
plt.title('Equalized image histogram')
plt.xlim([0, 256])
plt.ylim(0, 15000)

plt.show()

In [ ]:
# Load the original image in grayscale
image_path = '/content/drive/MyDrive/PIM/Images/X-ray_1.jpeg'
original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Simulates a low contrast image derived from the original image
low_contrast_image = cv2.normalize(original_image, None, alpha=75, beta=200, norm_type=cv2.NORM_MINMAX)

# Apply histogram equalization to the low-contrast image
equalized_image = cv2.equalizeHist(low_contrast_image)

# Apply contrast stretching to the low-contrast image (stretched to the full 0-255 range)
stretched_image = cv2.normalize(low_contrast_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

# Display the original, mapped, equalized, and stretched images
plt.figure(figsize=(12, 6))

# Plot the original image
plt.subplot(2, 4, 1)
plt.imshow(original_image, cmap='gray', vmin=0, vmax=255)
plt.title('Original image')
plt.axis('off')

# Plot the low-contrast image
plt.subplot(2, 4, 2)
plt.imshow(low_contrast_image, cmap='gray', vmin=0, vmax=255)
plt.title('Low-contrast image')
plt.axis('off')

# Plot the histogram-equalized image
plt.subplot(2, 4, 3)
plt.imshow(equalized_image, cmap='gray', vmin=0, vmax=255)
plt.title('Histogram equalized image')
plt.axis('off')

# Plot the contrast-stretched image
plt.subplot(2, 4, 4)
plt.imshow(stretched_image, cmap='gray', vmin=0, vmax=255)
plt.title('Contrast stretched image')
plt.axis('off')

# Plot the histogram of the original image
plt.subplot(2, 4, 5)
plt.hist(original_image.flatten(), bins=256, range=[0, 256], color='black')
plt.title('Original image\n histogram')
plt.xlim([0, 256])

# Plot the histogram of the low-contrast image
plt.subplot(2, 4, 6)
plt.hist(low_contrast_image.flatten(), bins=256, range=[0, 256], color='black')
plt.title('Low-contrast image\n histogram')
plt.xlim([0, 256])

# Plot the histogram of the histogram-equalized image
plt.subplot(2, 4, 7)
plt.hist(equalized_image.flatten(), bins=256, range=[0, 256], color='black')
plt.title('Histogram equalized\n image histogram')
plt.xlim([0, 256])

# Plot the histogram of the contrast-stretched image
plt.subplot(2, 4, 8)
plt.hist(stretched_image.flatten(), bins=256, range=[0, 256], color='black')
plt.title('Contrast stretched\n image histogram')
plt.xlim([0, 256])

# Adjust layout to prevent overlap and show the plot
plt.tight_layout()
plt.show()

### Contrast Limited Adaptive Histogram Equalization (CLAHE)

In [ ]:
# Load the original image in grayscale
image_path = '/content/drive/MyDrive/PIM/Images/X-ray_1.jpeg'
original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Create a CLAHE object
clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8, 8))

# Apply CLAHE to the original image
clahe_image = clahe.apply(original_image)

plt.figure(figsize=(8, 8))

# Plot the original image
plt.subplot(1, 2, 1)
plt.imshow(original_image, cmap='gray', vmin=0, vmax=255)
plt.title('Original image')
plt.axis('off')

# Plot the CLAHE-enhanced image
plt.subplot(1, 2, 2)
plt.imshow(clahe_image, cmap='gray', vmin=0, vmax=255)
plt.title('CLAHE image')
plt.axis('off')

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

In [ ]:
# Compares CLAHE, histogram equalization and contrast stretching applied to the low contrast image

# Create a CLAHE object (Arguments are optional)
clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8, 8))

# Apply CLAHE to the original image
clahe_image = clahe.apply(low_contrast_image)

# Define the images and titles
images = [low_contrast_image, clahe_image, equalized_image, stretched_image]
titles = ['Low contrast image', 'CLAHE', 'Histogram equalized image', 'Contrast stretched image']

# Plot all images using plt.subplot
plt.figure(figsize=(8, 8))

# Loop through the images and titles to display each one
for i in range(len(images)):
    plt.subplot(2, 2, i + 1)  # Create a 2x2 grid of subplots
    plt.imshow(images[i], cmap='gray', vmin=0, vmax=255)
    plt.title(titles[i])
    plt.axis('off')

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

In [ ]:
# Recursive CLAHE

# Create a CLAHE object
clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8, 8))

# Apply CLAHE up to three times
clahe_image_1 = clahe.apply(low_contrast_image)
clahe_image_2 = clahe.apply(clahe_image_1)
clahe_image_3 = clahe.apply(clahe_image_2)

# List of images and titles
images = [low_contrast_image, clahe_image_1, clahe_image_2, clahe_image_3]
titles = ['Low contrast image', 'CLAHE applied once', 'CLAHE applied twice', 'CLAHE applied thrice']

# Display the images in a 1x4 grid (first row)
plt.figure(figsize=(12, 6))

for i in range(4):
    plt.subplot(2, 4, i + 1)
    plt.imshow(images[i], cmap='gray', vmin=0, vmax=255)
    plt.title(titles[i])
    plt.axis('off')

# Display the corresponding histograms in a 1x4 grid (second row)
for i in range(4):
    plt.subplot(2, 4, i + 5)
    plt.hist(images[i].flatten(), bins=256, range=[0, 256], color='black')
    plt.title(f'{titles[i]} histogram')
    plt.xlim([0, 256])

plt.tight_layout()
plt.show()

## 3.- Bit-Plane Slicing

In [ ]:
# Load the image in grayscale
image_path = '/content/drive/MyDrive/PIM/Images/X-ray_1.jpeg'
original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Extract and display the 8 bit-planes
plt.figure(figsize=(10, 5))
plt.suptitle('Bit-plane slicing')

for i in range(8):
    # Extract the specific bit-plane
    bit_plane_image = (original_image >> i) & 1  # Shift to extract the i-th bit and mask to get that bit
    # After 8 iterations, all 8 bit planes (from the 0th bit to the 7th bit) have been extracted
    bit_plane_image *= 255  # Scale to full intensity range

    # Display the bit-plane image in the appropriate subplot
    plt.subplot(2, 4, i + 1)
    plt.imshow(bit_plane_image, cmap='gray')
    plt.title(f'Bit-plane {i}')
    plt.axis('off')

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

In [ ]:
# Prepare the figure for plotting
plt.figure(figsize=(12, 6))
plt.suptitle('Bit-plane reconstruction')

# Initialize an empty image to accumulate the bit-planes
accumulated_image = np.zeros_like(original_image, dtype=np.uint8)

# Loop through from the 7th bit-plane (MSB) down to the 0th bit-plane (LSB)
for i in range(7, -1, -1): # values from 7 to 0
    # Extract the specific bit-plane and add it to the accumulated image
    bit_plane_image = (original_image >> i) & 1  # Extract the i-th bit
    accumulated_image += bit_plane_image << i    # Add the bit-plane back to
                                                 # its original place in the image

    # Display the accumulated image after adding each bit-plane
    plt.subplot(2, 4, 8 - i)
    plt.imshow(accumulated_image, cmap='gray', vmin=0, vmax=255)
    plt.title(f'7th to {i}th bit-plane')
    plt.axis('off')

# Adjust layout and display the plot
plt.tight_layout()
plt.show()